# Functions (advanced)

# Table of Contents

- [Recap](#Recap)
  - [Functions are objects](#Functions-are-objects)
  - [Scopes and namespaces](#Scopes-and-namespaces)
- [Mutable objects as default values of function's parameters](#Mutable-objects-as-default-values-of-function's-parameters)
- [Lambdas](#Lambdas)
  - [Lambdas and sorting](#Lambdas-and-sorting)
- [Closures](#Closures)
  - [Modifying the free variable](#Modifying-the-free-variable)
  - [Multiple instances of closures](#Multiple-instances-of-closures)
  - [Closures can be tricky](#Closures-can-be-tricky)
  - [Nested closures](#Nested-closures)
  - [Closures: examples](#Closures:-examples)
    - [Example 1](#Example-1)
    - [Example 2](#Example-2)
- [Decorators](#Decorators)
  - [Decorators: examples](#Decorators:-examples)
    - [Example 1: timer](#Example-1:-timer)
      - [Fibonacci with recursion](#Fibonacci-with-recursion)
      - [Fibonacci with a simple loop](#Fibonacci-with-a-simple-loop)
      - [Fibonacci using `reduce`](#Fibonacci-using-reduce)
    - [Example 2: memoization](#Example-2:-memoization)
  - [Parametrized decorators](#Parametrized-decorators)
- [Generators](#Generators)
  - [Create an interable from a generator](#Create-an-interable-from-a-generator)
  - [Combining generators](#Combining-generators)
- [Exercises](#Exercises)
  - [Password checker factory](#Password-checker-factory)
  - [String range](#String-range)
  - [Read `n` lines](#Read-n-lines)
  - [Only run once](#Only-run-once)

We are going to cover the following topics:

- Decorators
- Lambdas
- Arguments and object's mutability
- Generators

## Recap

Before starting our deep dive on functions, we must revise quickly two important concepts. Have a look at the [Functions](./functions.ipynb#The-scope-of-a-function) notebook for more detail.

1. Scopes and namespaces
2. Functions are objects

### Functions are objects

As an example, suppose that we want to create a "password checker", that is, a function that can verify if an input password complies with some rules (e.g., minumum length, a given number of special characters). We could create a function with the following signature:

```python
def check_password(
    password: str,
    min_length: int,
    min_uppercase: int,
    min_punctuation:
    int, min_digits: int
    ) -> bool:
    """Check if a given password complies with pre-defined rules."""
```

In various situations, passwords are subject to distinct rules. Once these rules are defined, our goal is to streamline the process of handling them. We aim to avoid repeatedly inputting them for every password to check, as this can become tedious.

We can instead define a so-called **higher-order function** (see [Functional programming](./functional_programming.ipynb#Higher-Order-Functions-/-Functions-as-Values)): a function that returns another function.
It does **not** call that function, just returns it.

```python

def check_password_factory(
    min_length: int,
    min_uppercase: int,
    min_punctuation: int,
    min_digits: int
    ):
    """Our password checker factory"""

    def check_password(password: str) -> bool:
        """Password checker function"""
        # our password checking logic
        # ...
        return # True or False

    return check_password
```

You would first call your factory function with some password requirements:

```python
password_checker = check_password_factory(min_length=10, min_uppercase=4, min_punctuation=3, min_digits=1)
```

And then you could verify that an input password adheres to the constraints:

```python
password_checker("MyveryComplexPWD123")
```

### Scopes and namespaces

Python's variables are just names (i.e., labels) that we can **bind** to objects. Each variable is simply telling Python where to look in our computer's memory to retrieve some data. These bindings are **not global**: some of them exist only in specific parts of our code.

> The portion of code where a name binding is defined is called **lexical scope** (or just "scope"). The bindings are stored in a scope's **namespace**

We always have the following scopes:

1. `built-in` scope
2. `global` (or module) scope

We also have the `local` scope that's created when we are **calling** a function.
The local scope associated to any called function is **destroyed** after the function has done its job. Also the namespace associated with it will be gone.

When Python needs to retrieve which object is referenced by a given name, it always starts from the current scope (the `local` one if we are inside a function's body). If a name binding is not found there, it searches in the scope immediately up in the hierarchy.

> **LEGB rule**: **L**ocal → **E**nclosing → **G**lobal → **B**uilt-in

When Python encounters a function **definition** (i.e., at compile-time), it does two things:

1. Scans for any variables that have values **assigned** anywhere in the function. By default, names that are assigned are **local** unless we are explicitly saying that they should not with the `global` keyword.
2. Variables that are **referenced** but **not assigned** a value anywhere in the function will **not be local**. When we are calling the function (i.e., run-time), Python will look for them in the **enclosing scope**.

Examples:

```python
var = 10   # global (or module) scope

def func_1():
    print(var)   # var is referenced but not assigned. At compile-time is "non-local"

def func_2()
    var = 100    # var is assigned. At compile-time will be placed in the "local" scope

def func_3():
    global var
    var = 1000   # var is assigned, so it should be local. But it's also declared to be "global" with the keyword above

def func_4():
    print(var)
    var = 100    # what happens here?
```

A function gets its local scope upon calling. Since we can have function definitions inside of other functions, there can be **nested scopes**. This is where the `nonlocal` keyword becomes useful or even needed.

> The `nonlocal` keyword is used to declare that a variable is not local to the current function but is defined in the **nearest enclosing scope** that is **not global**. It allows you to access and modify variables in the outer (non-global) scope from within an inner function.

An example:

In [ ]:
def outer_function():
    x = "global"

    # prints "outer"
    print("Inside outer_function:", x)

    def inner_function():
        nonlocal x
        x = "inner"
        # prints "inner"
        print("Inside inner_function:", x)

    inner_function()
    # prints "inner" again, because we modified `x` from a nested scope
    print("Inside outer_function:", x)

outer_function()

Two important notes about the `nonlocal` keyword:

1. Python will search for a `nonlocal` name in the **enclosing local scopes** until it first encounters the specified variable.
2. **Only** local scopes are searched, never the global one.

<div class="alert alert-block alert-info">
    <h4><b>Hint</b></h4> <strong>Experiment</strong> a bit with the code above. If you didn't fully understand what <code>global</code> and <code>nonlocal</code> do, try changing the scope of <code>x</code> with the different keywords and see whether you obtain what you expect.
</div>

## Mutable objects as default values of function's parameters

Using mutable objects like lists or dictionaries as a function's parameters default values requires extra care, as it can lead to some unexpected or unwanted behaviors.
They could even produce mistakes that are difficult to debug.

Consider what happens when Python evaluates the following code:

```python
def func(a=10):
    return a ** 2
```

A new function is **created**, but it's body is run only when **executed**.
Also, a local scope for this function is only created upon its execution.

What happens at "compile-time" is setting the default value of the parameter `a`.
Where's the problem, you might say? Look at this example:

In [ ]:
from datetime import datetime

def log(msg, *, dt=datetime.utcnow()):
    print(f'{dt}: {msg}')

A simple logging function.
Let's run it a few times, waiting a bunch of seconds in between:

In [ ]:
from time import sleep

log("my first message")
sleep(5)
log("my first message")

Something is wrong here: we waited 5 seconds, but the timestamp of our log message **did not change**.
Why? Because `dt` default value is set **when the function is defined**, and it's never changed afterwards – unless we pass it by ourselves.

The correct pattern to avoid this unwanted behavior is setting a default value of `None`, so that the argument remains optional.
Then, inside the function's body, we can assign the correct or updated value:

In [ ]:
def log(msg, *, dt=None):
    dt = dt or datetime.utcnow()
    print(f'{dt}: {msg}')

In [ ]:
log("my first message")
sleep(5)
log("my first message")

Now the output is what we expected.

Another problematic context is when we're dealing with **mutable objects** (e.g., lists, sets, dictionaries).
This definition includes custom classes, if we are not careful.

Let's consider this example: we want to keep track of our groceries in different stores.
We might create a function that adds an item to a grocery list:

In [ ]:
def add_item(name, quantity, unit, grocery_list):
    grocery_list.append(f"{name} ({quantity} {unit})")
    return grocery_list

We now have two stores and want to add some items to them:

In [ ]:
store_1 = []
store_2 = []

add_item('bananas', 2, 'units', store_1)
add_item('grapes', 1, 'bunch', store_1)
add_item('python', 1, 'medium-rare', store_2)

print(store_1, store_2)

All good.
What if we don't supply the store list where we want to add the new item?
We could have our function create a new, empty store list:

In [ ]:
def add_item(name, quantity, unit, grocery_list=[]):
    grocery_list.append(f"{name} ({quantity} {unit})")
    return grocery_list

In [ ]:
store_1 = add_item('bananas', 2, 'units')
add_item('grapes', 1, 'bunch', store_1)

print(store_1)

Okay, all good.
Let's create our second list:

In [ ]:
store_2 = add_item('milk', 1, 'gallon')

print(store_2)

Again, not what we expected, right?
`store_2` should be a completeley new list, while Python is still adding to the empty list used to initialize the default value of the `grocery_list` parameter.

The solution pattern is similar to the logging function:

In [ ]:
def add_item(name, quantity, unit, grocery_list=None):
    if not grocery_list:
        grocery_list = []
    grocery_list.append(f"{name} ({quantity} {unit})")
    return grocery_list

In [ ]:
store_1 = add_item('bananas', 2, 'units')
add_item('grapes', 1, 'bunch', store_1)

In [ ]:
store_1

In [ ]:
store_2 = add_item('milk', 1, 'gallon')
store_2

And now everything works as we expected.

Using mutable objects as default values usually lead to unwanted results.
But there are some cases when this is precisely what we want.

As an example where this might be useful, consider a function to calculate the factorial:

In [ ]:
def factorial(n):
    if n < 1:
        return 1
    else:
        print(f'Calculating {n}!')
        return n * factorial(n-1)

In [ ]:
factorial(3)

In [ ]:
factorial(6)

We had to recalculate some values the second time, values that we could have saved for any subsequent call.
We will see a much better approach later on, but now consider the following `factorial` function:

In [ ]:
def factorial(n, cache={}):
    if n < 1:
        return 1
    elif n in cache:
        return cache[n]
    else:
        print(f'Calculating {n}!')
        result = n * factorial(n-1)
        cache[n] = result
        return result

In [ ]:
factorial(3)

In [ ]:
factorial(3)

In [ ]:
factorial(6)

Since the dictionary `cache` is initialized as an empty `dict()` **at compile-time** (when we define the function), we can update its content in any subsequent call to `factorial`.
This is an efficient way of reducing the run-time of a computation when we know we can store previously computed results.

## Lambdas

We already now how to create a function: we use the `def` keyword:

```python
def mult(a, b):
    return a * b
```

A function can have **parameters** and a `return` statement.
If we don't write a `return` statement, Python adds one for us and returns the `None` object.

There's another way to define a function object: with **lambda expressions** (or lambdas).
The syntax is similar with a few differences:

```python
lambda x: x ** 2
```

- We don't have `def`
- The function has **no name**
- There is **no `return` statement**

The "body" of a lambda expression follows the `:` mark.
If the expression evaluates to something, the lambda implicitly returns that value.

Lambdas are objects, as much as functions are, so we can define a lambda and assign it to a name:

In [ ]:
f = lambda x: x ** 2

In [ ]:
f

We can also define lambdas with parameters **with a default value**:

In [ ]:
g = lambda x, y=10: (x, y)

In [ ]:
g(10)

In [ ]:
g(10, -10)

Lambdas are very handy when we need something that behaves like a function, but we don't plan to use it many times. Examples:

```python

lambda x: x ** 2
lambda x, y: x + y
lambda: 'hello!'          # no params
lambda s: s[::-1].upper() # what does it do?
```

Lambdas are **anonymous function objects**.

In [ ]:
type(g)

Since are objects, they can be passed to (or returned from) other functions:

In [ ]:
def apply_func(fn, *args, **kwargs):
    return fn(*args, **kwargs)

In [ ]:
apply_func(lambda x, y: x+y, 1, 2)

In [ ]:
apply_func(lambda *args: sum(args), 1, 2, 3, 4, 5)

The previous example is **not** the suggested way to sum values of an iterable: we should use the built-in `sum()` when appropriate:

In [ ]:
apply_func(sum, (1, 2, 3, 4, 5))

In [ ]:
sum((1, 2, 3, 4, 5))

### Lambdas and sorting

Python has a built-in `sorted` method returns any iterable sorted according to a default ordering.
Sometimes you may want to (or need to) specify a different criteria for sorting.

In [ ]:
letters = list("ABCDzywab")

In [ ]:
sorted(letters)

Python's `sorted` has a keyword-only argument named `key=` that takes a function used to return the key – that is, the ordinal criteria according to which we want to sort the iterable.

In [ ]:
sorted(letters, key=str.upper)  # sort as if all letters are CAPITALIZED

Let's see how we can created a "sorted dictionary".
Recall that a dictionary is an **unordered collection**, so it doesn't have a built-in order.

_(Well, that's not completely true. The most recent versions of Python store the key-value pairs in the order they are entered or supplied.
The thing is: accessing a dictionary **by index** doesn't make sense.)_

In [ ]:
d = {'def': 300, 'abc': 200, 'ghi': 100}
sorted(d)

Iterating over a dictionary is equivalent to iterate **over keys**.
If we wanted to sort our dictionary by its values, we need to use a lambda:

In [ ]:
sorted(d, key=lambda k: d[k])

But wait: now we lost our values!
We need to do something more elaborate if we want to have a dictionary back:

In [ ]:
dict(sorted(d.items(), key=lambda x: x[1]))

Another useful application of lambdas is when Python doesn't know how to apply an ordering to some kind of data.
For example, with complex numbers:

In [ ]:
complex = [3+3j, 1+1j, 0, 4-2j]

In [ ]:
sorted(complex)

We can sort complex numbers based on their modulus:

In [ ]:
sorted(complex, key=lambda z: (z.real)**2 + (z.imag)**2)

<div class="alert alert-block alert-warning">
    <h4><b>Question</b></h4> Can you find a way to <strong>randomize</strong> a list using <code>sorted</code> and lambdas?
</div>

<div class="alert alert-block alert-info">
    <h4><b>Hint</b></h4> Have a look at the <a href="https://docs.python.org/3/library/random.html"><code>random</code> module</a> of Python's standard library.
</div>

## Closures

Let's consider the following code:

In [ ]:
def outer():
    lang = "Python"

    def inner():
        print(f"{lang} rocks!")

    inner()

outer()

Here the `lang` variable is **non-local** to `inner()` because it's only referenced. `lang` is also called **free variable**.

> A **free variable** is a variable referenced locally but defined in the enclosing scope.

Also, `lang` and `inner()` both belongs to the local scope of `outer()`. Since this bound is particularly special, it has a special name: it's called a **closure**.
The name "closure" come from the function `inner()` _enclosing_ its free variable `lang`.

Let's make a small adjustment that will change a lot of things:

In [ ]:
def outer():
    lang = "Python"

    def inner():
        print(f"{lang} rocks!")

    return inner

outer()

We turned `outer()` into an higher-order function that does not return a simple function, but a closure (`inner()` + the free variable).

Since functions are objects, we can assign that to a name:

In [ ]:
fn = outer()

And then call that function as any other function:

In [ ]:
fn()

But wait a second! How's that possible? 🤔

`fn()` is called **after** `outer()` has run: it runs when we are assigning the name `fn` to the result of calling `outer()`.
If the local scope of a function is destroyed after the function has run, how can `fn` know that `lang = "Python"`?

That's because Pyhon realized that we created a closure, and it's doing something unusual.

If we look once again to the example above, we see that the name `lang` is **shared** between the local scope of `outer()` and the `print` statement inside `inner()`.
When Python sees this, it does something different: it creates an **intermediary** object – called a _cell object_ – that only contains a memory address.
A memory address of what? Of whatever object (i.e. data) is assigned to `lang`, the free variable.

![](./images/cell_object.png)

We can see all that by inspecting some _hidden_ attributes of `fn`:

In [ ]:
fn.__code__.co_freevars

In [ ]:
fn.__closure__

We can see now the reason why we can call `fn()` and see the string "Python rocks!" printed although the variable `lang` is now out of scope (it's been destroyed).
There is another reference to the cell object, that from the closure created by `inner()` plus the free variable.
When running `outer()`, `inner()` is **not called**, and Python still knows how to retrieve the value of the string object.

### Modifying the free variable

We know that the `nonlocal` keyword allows us to modify variables from the **enclosing scope**. Therefore, the following closure will work as expected:

In [ ]:
def counter():
    count = 0 # local variable

    def inc():
        nonlocal count
        count += 1
        return count
    return inc

c = counter()

c()

The `inc()` function and the `count` variable are the closure, but the `count` variable is not only accessed, but also modified.

We can also have multiple closures that reference (and modify) the same free variable:

In [ ]:
def adders():
    count = 0

    def add_1():
        nonlocal count
        count += 1
        return count

    def add_2():
        nonlocal count
        count += 2
        return count

    return add_1, add_2

fn1, fn2 = adders()

In [ ]:
fn1()

In [ ]:
fn2()

### Multiple instances of closures

As we saw before when talking about scopes, every time a function is **called** a new **local scope** is created.
A closure can then be created multiple times, and each time we are calling it a new **extended scope** is created.

Consider the following example:

In [ ]:
def power(n):
    # `n` is a local variable
    def op(x):
        return x ** n
    return op

`n` is our free variable, and we have a closure that contains the function `op()` and the free variable.

In [ ]:
square = power(2)
cube = power(3)

In [ ]:
print(square(10))
print(cube(10))

We can verify that the two closures are completely different even though they were created from the same `power()`:

In [ ]:
square.__closure__

In [ ]:
cube.__closure__

As we expected, the free variable `n` (of type `int`) is referenced by the cell objects of both closures.
And since a new value for `n` is created in the local namespace of `power()` every time it's called, we obtained two **different** `int` objects.

### Closures can be tricky

One important aspect of closures can be the source of nasty bugs if we don't understand it well.

> A free variable is **referenced** when the closure is created, but its value is **looked up** upon calling.

Let's say we want to create a function to sum different factors to an arbitrary number.
We could do:

In [ ]:
def adder(n):
    def op(x):
        return n + x
    return op

In [ ]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)

This works as expected:

In [ ]:
add_1(10), add_2(10), add_3(10)

Now we have the following idea to improve our code:

In [ ]:
def fancy_adders():
    adders = []
    for n in range(1, 5):
        def op(x):
            return x + n
        adders.append(op)
    return adders

In [ ]:
adders = fancy_adders()

We have now 4 functions in the `adders` list:

In [ ]:
adders

Let's see what happens when we call them:

In [ ]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

Wait, why?! It seems that we picked up the **same value** of the free variable.
The free variable is always `n`:

In [ ]:
adders[0].__code__.co_freevars

And we can indeed verify that its value referenced by each closure is exactly the same:

In [ ]:
[x.__closure__ for x in adders]

Which value? The last iteration of our loop, `n=4`.
In fact:

In [ ]:
hex(id(4))

The key to understand this behavior is remembering that **closures captures _variables_ and not _values_**.
This means that every `op()` function created in the loop is closing over the same variable `n`.

By the time the call to `fancy_adders()` is over, the value of `n` is incremented to its final value, that is, 4.
This is the value that will be looked up when calling our closures! And this is why we can see that `hex(id(4))` – the memory address of the integer `4` – is indeed the same for all closures.

If we wanted to fix the example as we intended, we need to capture the **current** value of `n` when defining the closure:

In [ ]:
def truly_fancy_adders():
    adders = []
    for n in range(1, 5):
        def op(x, n=n):  # Capture the current value of n as a default argument
            return x + n
        adders.append(op)
    return adders

In [ ]:
correct_adders = truly_fancy_adders()

In [ ]:
correct_adders[0](10), correct_adders[1](10), correct_adders[2](10)

Now, let's inspect our correct closures:

In [ ]:
[x.__closure__ for x in correct_adders]

Hey, why `None`? Let's check our free variable:

In [ ]:
correct_adders[0].__code__.co_freevars

Nothing yet?
Can you think why you don't get any output?

<div class="alert alert-block alert-info">
    <h4><b>Hint</b></h4> Think about <strong>your</strong> answer before evaluating the cell below.
</div>

In [ ]:
from tutorial.quizes.functions_advanced import tricky_closures as answer
answer

### Nested closures

We can also nest closures, as much as we can nest functions:

In [ ]:
def incrementer(n):
    def inner(start):
        current = start
        def inc():
            a = 10  # local variable, NOT a free variable
            nonlocal current
            current += n
            return current
        return inc
    return inner

In [ ]:
f = incrementer(2)
f.__code__.co_freevars

We create an incrementer function with the default increment, `n=2`, starting from `100`:

In [ ]:
inc_2 = f(100)
inc_2()

We can also create another _custom_ incrementer with a different increment value:

In [ ]:
inc_10 = incrementer(10)(100)
inc_10()

### Closures: examples

#### Example 1

Let's see a practical example of using closures.
We're going to see how closures can replace classes and be more straightforward for simple tasks.

Say that we want to calculate a running average of some numbers that we don't know in advance.
We could create a class as follows:

In [ ]:
class Averager:
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        return sum(self.numbers) / len(self.numbers)

a = Averager()

In [ ]:
a.add(10)

In [ ]:
a.add(20)

In [ ]:
a.add(30)

How can we rewrite the our class as a closure?
The free variable will be the list `numbers`.

In [ ]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        return sum(numbers) / len(numbers)
    return add

a = averager()

In [ ]:
a.__closure__

In [ ]:
a.__code__.co_freevars

In [ ]:
a(10)

In [ ]:
a(20)

We can make it better: instead of accumulating all the numbers in a list, we only need to keep a running total and count.

In [ ]:
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total
        nonlocal count
        
        total += number
        count += 1
        return total / count
    return add

a = averager()

In [ ]:
a(10)

In [ ]:
a(20)

In [ ]:
a(30)

#### Example 2

We want to create a counter function, that is, a function that increments a variable every time it's called:

In [ ]:
def counter(initial_value):
    def inc(increment=1):
        nonlocal initial_value
        initial_value += increment
        return initial_value
    return inc

In [ ]:
c1 = counter(0)
c100 = counter(100)

In [ ]:
c1()

In [ ]:
c100()

In [ ]:
c1(2)

In [ ]:
c100(10)

As you can see, each closure maintains a reference to the `initial_value` variable that was created when the `counter()` function was **called**.

Each time that function was called, a new local variable `initial_value` was created (with a value assigned from the argument), and it became a nonlocal (captured) variable in the inner scope.

Let's extend this example to a **function counter**: a counter that keeps track how many times a function is run.

In [ ]:
def fcounter(function):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"Function '{function.__name__}' has beel called {count} times.")
        return function(*args, **kwargs)

    return inner

Let's define a function we want to keep track of:

In [ ]:
def add(a, b):
    return a + b

counter_add = fcounter(add)

In [ ]:
counter_add.__code__.co_freevars

We have **two** free variables, one of which is a function (remember: functions are objects).

In [ ]:
counter_add(1, 2)

In [ ]:
counter_add(2, 3)

## Decorators

Now that we know what closures are and what we can do with them, understanding what is a decorator is a (small) step away.

Consider again the last example in the previous section: a counter for functions.

In [ ]:
def fcounter(function):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"Function '{function.__name__}' has beel called {count} times.")
        return function(*args, **kwargs)

    return inner

And then create a function we want to keep track:

In [ ]:
def factorial(n):
    product = 1
    if n == 0:
        return 1
    for i in range(2, n+1):
        product *= i
    return product

In [ ]:
counter_fact = fcounter(factorial)

In [ ]:
counter_fact.__closure__

In [ ]:
counter_fact(10)

Of course, `counter_fact` is an arbitrary name.
Nothing prevents us from calling it `factorial`:

In [ ]:
factorial = fcounter(factorial)

In [ ]:
factorial(10)

This way of defining a function, creating a closure that "wraps" a function object, and then **renaming** the initial function is so common in Python that gained a special syntax:

In [ ]:
@fcounter
def mult(a: float, b: float) -> float:
    """Multiplies two floats"""
    return a * b

In [ ]:
mult(2.0, 4.0)

The function `fcounter` is called a **decorator**, because it's placed **before** the function definition line with the special symbol `@`.

There's one problem, though.
If we inspect our `mult` function, we could see that it has lost something:

In [ ]:
mult.__name__

In [ ]:
help(mult)

As you can see, we've also lost our docstring and type hints!
What's left is the docstring and the type annotations of the `inner` function.

In [ ]:
import inspect

print(inspect.getsource(mult))

In [ ]:
print(inspect.signature(mult))

We _could_ put back that information, but it might not be straighforward:

In [ ]:
def fcounter(function):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"Function '{function.__name__}' has beel called {count} times.")
        return function(*args, **kwargs)

    inner.__name__ = function.__name__
    inner.__doc__ = function.__doc__

    return inner

In [ ]:
@fcounter
def add(a: int, b: int = 10) -> int:
    """Sum two integers"""
    return a + b

In [ ]:
help(add)

In [ ]:
add.__name__

Okay, at least our docstring and function's name are back.
What about the type annotations?

In [ ]:
inspect.signature(add).parameters

Unfortunately, they stil belong to the `inner` function.
There's a way to bring them back, and we have to use a built-in function from the `functolls` module called `wraps`.

Curiously, `functools.wraps` is **itself a decorator**!

In [ ]:
from functools import wraps

def fcounter(function):
    count = 0

    @wraps(function)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"Function '{function.__name__}' has beel called {count} times.")
        return function(*args, **kwargs)

    return inner

In [ ]:
@fcounter
def add(a: int, b: int = 10) -> int:
    """Sum two integers"""
    return a + b

In [ ]:
help(add)

In [ ]:
inspect.signature(add)

In [ ]:
inspect.signature(add).parameters

And now everything is back to normal.

### Decorators: examples

#### Example 1: timer

This is classic example of using decorators: creating a timer for a generic function.

In [ ]:
from time import perf_counter
from functools import wraps

def timed(function):

    @wraps(function)
    def inner(*args, **kwargs):
        start = perf_counter()
        
        result = function(*args, **kwargs)
        
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        
        print(f'{function.__name__}({args_str}) took {elapsed:.6f}s to run.')

        return result
    
    return inner

Let's test it with a function to calculate the n-th Fibonacci number: `1, 1, 2, 3, 5, 8, 11, ...`

We're going to write **three** Fibonacci implementations to compare their efficiency:

1. With recursion
2. With a simple loop
3. A functional approach

**NOTE**: while Python indexes start from 0, our Fibonacci sequence starts from 1 (by choice).

##### Fibonacci with recursion

In [ ]:
def calc_fib_recursive(n):
    if n <= 2:
        return 1
    return calc_fib_recursive(n - 1) + calc_fib_recursive(n - 2)

In [ ]:
calc_fib_recursive(3)

In [ ]:
calc_fib_recursive(6)

In [ ]:
@timed
def fib_recursive(n):
    return calc_fib_recursive(n)

In [ ]:
fib_recursive(33)

In [ ]:
fib_recursive(35)

In [ ]:
fib_recursive(40)

Sounds a bit long, doesn't it?
Well, it's: we are calculating the same numbers **every time**.
When we're past the 30th number, we start seeing some considerable slow down.

##### Fibonacci with a simple loop

In [ ]:
@timed
def fib_loop(n):
    fib_1 = 1
    fib_2 = 1
    
    for i in range(3, n + 1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    
    return fib_2


In [ ]:
for n in (3, 10, 30, 35, 40):
    fib_loop(n)

Incredibly more efficient!
This is just getting rid of multiple (useless) calculations.

##### Fibonacci using `reduce`

First, a quick refresher:

In [ ]:
from functools import reduce

In [ ]:
reduce(lambda x, y: x + y, (1, 2, 3, 4, 5))

It's just the progressive sum of pairs of numbers.
`reduce` applies an operation (1st argument) to pairs of element in an interable (2nd argument).

To calculate the Fibonacci sequence with `reduce`:

```
n=1:
(1, 0) --> (1, 1)

n=2:
(1, 0) --> (1, 1) --> (1 + 1, 1) = (2, 1)  : result = 2 

n=3
(1, 0) --> (1, 1) --> (2, 1) --> (2+1, 2) = (3, 2)  : result = 3

n=4
(1, 0) --> (1, 1) --> (2, 1) --> (3, 2) --> (5, 3)  : result = 5
```

In general each step in the reduction is as follows:

```
previous value = (a, b)
new value = (a+b, a)
```

If we start our reduction with an initial value of `(1, 0)`, we need to run our "loop" `n` times.
We therefore use a "dummy" sequence of length `n` to create `n` steps in our reduce.

In [ ]:
@timed
def fib_reduce(n):
    initial = (1, 0)
    fib_n = reduce(lambda prev, n: (prev[0] + prev[1], prev[0]), range(n), initial)
    return fib_n[0]

In [ ]:
for n in (3, 10, 30, 35, 40):
    fib_reduce(n)

If we compare the three methods:

In [ ]:
fib_recursive(35)
fib_loop(35)
fib_reduce(35)

Although the recursive method is the __easiest__ to understand, it's also the slowest because it's written inefficiently.
How can we improve it? Let's see a second example of using decorators.

#### Example 2: memoization

The previous example showed one task that a decorator can accomplish pretty well: adding some feature to a predefined function.
But what about __changing__ the behavior of the function itself?

Remember the Fibonacci sequence example.
We discovered that the recursive approach is by far the most intuitive, yet it's tremendously inefficient because a number gets calculated multiple times.

In [ ]:
def fib(n):
    print (f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(5)

You can see that `fib(2)` is calculated **three times**.
And the larger the number, the more often a number is recalculated.
That's why with `fib(40)` the recursive approach is taking ages to finish.

We'll see how we can improve this approach using a decorator and a caching mechanism for previously calculated numbers.
This approach is well-known in computer science, and it's called **memoization**.

For the sake of comparison, let's first approach this problem with a simple class:

In [ ]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1} # initial values already known
    
    def fib(self, n):
        if n not in self.cache:
            print(f'Calculating fib({n})')
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

In [ ]:
f = Fib()
f.fib(10)

In [ ]:
f.fib(12)

You can see that numbers $\leq 10$ are **not** recalculated, but are fetched from the cache.

Let's see how we can do this with a closure:

In [ ]:
def fib():
    # `cache` is our free variable
    cache = {1: 1, 2: 2}
    
    def calc_fib(n):
        if n not in cache:
            print(f'Calculating fib({n})')
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    
    return calc_fib

In [ ]:
f = fib() # create our closure
f(10)     # call it

In [ ]:
f(15)

Once again, cached valued are just returned and not recalculated.

How can we implement this as a decorator?

In [ ]:
from functools import wraps

def memoize_fib(fn):
    cache = {}
    
    @wraps(fn)
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    
    return inner

In [ ]:
@memoize_fib
def fib(n):
    print (f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(3)

In [ ]:
fib(10)

In [ ]:
fib(6)

`fib(6)` was literally instantaneous because we already had it in the cache.

How to create a generic decorator that caches the return values of **any** function?
We know how to do it:

In [ ]:
def memoize(fn):
    cache = {}
    
    @wraps(fn)
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args)
        return cache[args]
    
    return inner

And we can now give any function a cache to store previously calculated results:

In [ ]:
@memoize
def fact(n):
    print(f'Calculating {n}!')
    return 1 if n < 2 else n * fact(n-1)

In [ ]:
fact(6)

In [ ]:
fact(10)

In [ ]:
fact(9)

Caching and decorators play a crucial role in optimizing function performance.
By caching previously calculated results in memory (or on disk), we can drastically reduce the time required for the calculation.

However, our simple memoizer has a limitation: the cache size is **unbounded**, which may not be ideal.
In practice, it's often desirable to restrict the cache to a specific number of entries.
This helps strike a balance between computational efficiency and memory utilization.

Additionally, our current implementation does not handle keyword arguments (`**kwargs`), which can be a significant limitation in more complex scenarios.

Fortunately, Python provides a built-in solution for memoization in the `functools` module, known as `lru_cache`.
This decorator is designed to address the drawbacks of our basic memoization example.
`lru_cache` stands for **Least Recently Used** caching, meaning that when the cache reaches its limit, the least recently used entries are automatically removed to make room for new ones.
This feature ensures efficient memory management while improving performance.

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache()
def fact(n):
    print(f"Calculating fact({n})")
    return 1 if n < 2 else n * fact(n-1)

In [ ]:
for n in (2, 5, 6, 10, 15, 8):
    print(fact(n))

Once again, the last value `fact(8)` was simply fetched from the cache.

Now let's see if we have improved on our recursive approach of calculating Fibonacci numbers.
Recall the naive implementation:

In [ ]:
from time import perf_counter

In [ ]:
def fib_no_memo(n):
    return 1 if n < 3 else fib_no_memo(n-1) + fib_no_memo(n-2)

In [ ]:
start = perf_counter()
result = fib_no_memo(35)
done = perf_counter() - start

print(f"result={result}, elapsed: {done}s")

In [ ]:
@lru_cache()
def fib_memo(n):
    return 1 if n < 3 else fib_memo(n-1) + fib_memo(n-2)

In [ ]:
start = perf_counter()
result = fib_memo(35)
done = perf_counter() - start

print(f"result={result}, elapsed: {done}s")

It's about **4 orders of magnitude** faster than the naive approach! 🔥
Let's time it again to see what happens:

In [ ]:
start = perf_counter()
result = fib_memo(35)
done = perf_counter() - start

print(f"result={result}, elapsed: {done}s")

In [ ]:
start = perf_counter()
result = fib_memo(35)
done = perf_counter() - start

print(f"result={result}, elapsed: {done}s")

Not the same time, but about the same order of magnitude.
It means that no extra calculation was needed.

You may have noticed that `lru_cache` was called an **empty list of arguments**, but it supports some.
One of them is the **cache size**: by default, it can hold up to **128 items**.
The best is to use powers of 2 for performance reasons, but you can change it to anything you want, including `None` for an **unbounded** cache (not recommended).

In [ ]:
@lru_cache(maxsize=8)
def fib(n):
    print(f"Calculating fib({n})")
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(8)

In [ ]:
fib(9)

In [ ]:
fib(1)

We had to recalculate `fib(1)` because when we called `fib(9)` the least recent item in the cache (the result of `fib(1)`) was evicted from the cache.

### Parametrized decorators

Here comes a natural question: what if I need to pass some argument to my decorator?
Think again of the `functools.lru_cache`: it takes one parameters, the cache size called `maxsize`.

Let's bring back our `timed` decorator and make a small change.
Instead of calculating the time of a **single run**, we want to calculate an **average** of, say, `10` runs:

In [ ]:
from time import perf_counter

def timed(fn):
    def inner(*args, **kwargs):
        total_elapsed = 0
        
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        
        avg_elapsed = total_elapsed / 10
        
        print(f'Avg runtime: {avg_elapsed:.6f}s')
        
        return result
    
    return inner

In [ ]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

@timed
def fib(n):
    return calc_fib_recurse(n)

In [ ]:
fib(30)

But what if I wanted to time this function **100 times**?
Or say that I have different functions that should be timed with a different number of repetitions?
It's not the best to have the value `10` hard-coded, right?

Let's change this:

In [ ]:
def timed(fn, num_reps):  
    def inner(*args, **kwargs):
        total_elapsed = 0
        
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        
        avg_elapsed = total_elapsed / num_reps
        
        print(f'Avg runtime: {avg_elapsed:.6f}s ({num_reps} reps)')
        return result
    
    return inner

In [ ]:
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib, 5)

In [ ]:
fib(28)

But wait: why did we use the fancy `@-` syntax?
The reason is simple: with `@` the decorating function (`timed` here) can only take a **single argument**, that is, the function to be decorated.

To fix this behavior we need to rethink of what `@` is doing.
Writing

```python
@timed
def my_func():
    pass
```

is equivalent to

```python
my_func = timed(my_func)
```

When called, `timed` returns the **inner closure**, where the original function is the free variable.


In [ ]:
fib.__closure__

In [ ]:
fib.__code__.co_freevars

So, for the syntax `@timed(10)` to work, where `10` is the number of repetition, `timed` should return **a decorator itself**, and not our closure.
In practice, the `timed` function is a **decorator factory**: something that's able to return a "parametrized" decorator.

In [ ]:
from functools import wraps
from time import perf_counter

def timed(num_reps=10):
    
    def decorator(fn):

        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            
            avg_elapsed = total_elapsed / num_reps
            
            print(f'Avg Run time: {avg_elapsed:.6f}s ({num_reps} reps)')
            return result
        
        return inner
    
    return decorator  

In [ ]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

@timed(10)
def fib(n):
    return calc_fib_recurse(n)

In [ ]:
fib(10)

In [ ]:
from functools import lru_cache

def calc_fact(n):
    return 1 if n < 2 else n * calc_fact(n-1)

@timed(20)
@lru_cache()
def fact(n):
    return calc_fact(n)

In [ ]:
fact(10)

And yes, you can **stack multiple decorators**! 😎

## Generators

The concept of generators is very much tied to that of "looping over some kind of container".
And we already used generators many time without realizing it.
The easiest example is a standard `for` loop over some range of integers:

```python
for i in range(10):
    # do something
```

The object that Python builds for us with `range(10)` is something very close to a generator. 

To understand generators, we first need to review what it means to be **iterable** and, more importantly, what is an **iterator**.

1. An **iterable** is any object that can return one item at time until there are no items left.
2. An **iterator** is an object that represents a stream of data and keeps track of the current position while processing the stream. It must implement two methods of the _iterator protocol_: `__next__` (returns the next element in the stream and advances the position) and `__iter__` (returns the iterator object itself)

Delving deep into iterators is out of the scope of this section, so we are going to show you a practical example of a class that implements the "iterator protocol".

Example: we want an iterator that build squares of successive integers.

In [ ]:
class Squares:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.i >= self.n:
            raise StopIteration

        self.i += 1
        return self.i ** 2

In [ ]:
for sq in Squares(5):
    print(sq)

We see that we can indeed loop over our custom `Squares` class.
How Python is able to do this?

1. The `__next__` method returns the next item, without going past the last one
2. We raise a special exception if we are at the last item (or past)
3. The `__iter__` method returns an instance of the class, meaning that the object itself _is_ an iterator

As you might have learned by now, we can implement some built-in behavior in our classes by using the so-called "special methods" or **dunder methods**: those with this naming schema `__method__`.

A few examples:

- The `len()` built-in can be defined with the `__len__` method
- The string returned by `str()` can be defined with the `__str__` method. The same goes for an object's representation with `repr()` (and `__repr__`)
- The `[]` (fetching an item by index from an ordered collection) can be defined with the `__getitem__` method

Python also has the built-in `next()` which does what you think it does: it takes an **iterator** object and returns the next element in the stream of data by calling the `__next__` method implemented by that object.

It the same way, we can call `iter()` on an object as the **only** argument and return an iterator.
Our class is doing that by implementing the `__iter__` method.

But there's another way of calling `iter()` with **two arguments**: the first must be a **callable** (i.e., a function) and the second argument is a **sentinel**. As soon as the callable returns the sentinel value, then a `StopIteration` is raised.

We could've written our `Squares` class using a closure instead:

In [ ]:
def square():
    i = 0
    def inner():
        nonlocal i
        i += 1
        return i ** 2
    return inner

square_iter = iter(square(), 5**2)

In [ ]:
for sq in square_iter:
    print(sq)

If the value returned by `square()` is 25 (our sentinel), then a `StopIteration` is raised.

These two ways are identical: in the first case (the class), we built the iterator ourselves. In the second case, Python built it for us.

The second example is a shorter code, but maybe a bit more difficult to understand if we didn't write it.
There's a better way to do the same, and it's using **generators** with their special keyword `yield`.

The `yield` statement is used almost like a `return` statement in a function, but there is a huge difference.
When the `yield` statement is encountered, Python returns whatever value `yield` specifies, but it **pauses** execution of the function.
We can then _call_ the same function again and it will _resume_ from where the last `yield` was encountered.

We do **not** resume the function by calling it the standard way, but we have to use the built-in `next()`:

In [ ]:
def my_func():
    print('line 1')
    yield 'Python'
    print('line 2')
    yield 'Is'
    print('line 3')
    yield 'Great'

In [ ]:
gen_my_func = my_func()
type(gen_my_func)

Here it is: our function returned _something_ different than the usual "function" object.
We did not run anything in the function body until we use it as an argument of `next()`:

In [ ]:
next(gen_my_func)

In [ ]:
next(gen_my_func)

In [ ]:
next(gen_my_func)

In [ ]:
next(gen_my_func)

A `StopIteration` is raised if we are trying to go past the last `yield` statement.
This should ring a bell: the `next()` method, a `StopIteration`... it seems that `gen_my_func` is very similar to an iterator.

How can we check it?
We know that an iterator **must** implement an `__iter__` method, right?

In [ ]:
'__iter__' in dir(gen_my_func)

And also the `__next__` method

In [ ]:
'__next__' in dir(gen_my_func)

We can also check that `iter()` applied on our object returns indeed the same thing.
That is, our object is itself an iterator.

In [ ]:
gen_my_func

In [ ]:
iter(gen_my_func)

Precisely the same object.

How Python knows when to stop the iteration?
When should it raise the `StopIteration`?
In the simple example above, it's easy: when there's nothing else after the last `yield`.

Well, not really "nothing". Remember that Python returns `None` for us if we don't specify any `return` statement.
So, in general, the iteration will terminate **when we return something from the function** using the `return` statement.

Let's go back to our `squares` example and refactor it to have a generator:

In [ ]:
def squares(sentinel):
    i = 0
    while True:
        if i < sentinel:
            yield i ** 2
            i += 1
        else:
            return 'Finished.'

In [ ]:
sq = squares(3)
next(sq)

In [ ]:
next(sq)

In [ ]:
next(sq) # this is the last

In [ ]:
next(sq) # a StopIteration is raised

Note how in the generator function above we incremented the number `i` **after** the `yield` statement.
That is, as soon as we resume our function, we make sure to be in the correct position of our _stream of data_ – in this case, a sequence of integers squared.

### Create an interable from a generator

As we know, generators are iterators.
This means that we can **consume** them (i.e., exhaust the elements they can return).
However, sometimes we want to create an interable instead, like a list, that we can loop over as many time as we want.

We know all the pieces to put together to obtain such a thing: we need a class that implements the iterator protocol.

Let's consider again the example of generating squares of integers:

In [ ]:
def squares_gen(n):
    for i in range(n):
        yield i ** 2

In [ ]:
sq = squares_gen(5)

In [ ]:
for num in sq:
    print(num)

But our generator is now exhausted and it has nothing left to return:

In [ ]:
next(sq)

To restart the iteration, we need to create another instance of the generator.
We can wrap this behavior in an **iterable class**:

In [ ]:
class Squares:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        return squares_gen(self.n)

In [ ]:
sq = Squares(5)
[num for num in sq]

And we can do it again:

In [ ]:
[num for num in sq]

We can put everything is a single class to make things easier to read:

In [ ]:
class Squares:
    def __init__(self, n):
        self.n = n
        
    @staticmethod
    def squares_gen(n):
        for i in range(n):
            yield i ** 2
        
    def __iter__(self):
        return Squares.squares_gen(self.n)

In [ ]:
sq = Squares(10)

In [ ]:
[num for num in sq]

In [ ]:
[num for num in sq]

### Combining generators

We have to be careful when using a generator with one another.
For example, the `enumerate()` built-in returns a generator to iterate over an indexed container.

In [ ]:
def squares(n):
    for i in range(n):
        yield i ** 2

In [ ]:
sq = squares(5)

In [ ]:
enum_sq = enumerate(sq)

Now, `enumerate` builds a generator itself, so `sq` had not been consumed yet at this point:

In [ ]:
next(sq)

In [ ]:
next(sq)

But since we now have consumed **2 elements** from `sq`, when we use `enumerate` it will also have two less items from `sq`:

In [ ]:
next(enum_sq)

And this might not be what you expected: the value is the **third** element of `sq` ($2^2$), while the index is `0`, as if we were starting from the beginning.
From the point of view of the generator returned by `enumerate`, **we are at the beginning**.

So, beware when you are combining multiple generators, and think carefully what's the behavior you expect.

## Exercises

In [ ]:
%reload_ext tutorial.tests.testsuite

### Password checker factory

Create a function called `password_checker_factory` that can be used to generate different password checkers.
This function will take **four parameters**: `min_uppercase`, `min_lowercase`, `min_punctuation`, and `min_digits`.
They represents the constraints on a given password:

1. The minimum number of uppercase letters.
2. The minimum number of lowercase letters.
3. The minimum number of punctuation characters.
4. The minimum number of digits.


The `create_password_checker` function generates another function that assesses a given password (string).
This resulting function returns a **tuple with two elements**:

1. The first element is a **boolean** indicating if the password passed validation.
2. The second element is a **dictionary** mapping `uppercase`, `lowercase`, `punctuation`, and `digits` to the difference between the actual count of each type in the password and its minimum requirement. Positive values denote exceeding, and negative values denote not meeting these minimums.

For example, to create a password checker that requires a password to have at least 2 uppercase letters, at least 3 lowercase letters, at least 1 punctuation mark, and at least 4 digits, we can write

```python
pc1 = create_password_checker(2, 3, 1, 4)
```

If we test the following passwords:

```python
print(pc1('Ab!1'))
print(pc1('ABcde!1234'))
```

We should get these results:

```python
(False, {'uppercase': -1, 'lowercase': -2, 'punctuation': 0, 'digits': -3})
(True, {'uppercase': 0, 'lowercase': 0, 'punctuation': 0, 'digits': 0})
```

In this example, the first password `Ab!1` is **invalid**: it lacks `1` uppercase character, `2` lowercase, and `3` digits.
Instead, the second password is **valid**.

In [ ]:
%%ipytest

def solution_password_checker_factory(min_up: int, min_low: int, min_pun: int, min_dig: int):
    """Password checker factory"""
    

### String range

Create a function called `str_range` that emulates the the built-in `range`, but for characters.
That is, when you call `str_range('j', 'm')`, you will get back a generator that produces each of the letters in between.

The function takes two **mandatory** parameters, `start` and `end`, plus an **optional** `step` value, with default value of `1`.

As opposed to Python's numeric `range()`, the string ranges generated by `str_range` are **including** their final string (that is, `end`).
Moreover, since Python 3 supports non-Latin characters (and even non-alphabetic), it should be possible to use any of them as a `start` or `end` value.

<div class="alert alert-block alert-info">
    <h4><b>Hint</b></h4> You might want to look up what it means to get an "integer representing the Unicode code point of that character". The <a href="https://docs.python.org/3/library/">official docs of Python</a> might help you.
</div>

<div class="alert alert-block alert-warning">
    <h4><b>Note</b></h4> It's okay if in some languages it doesn't quite exist the idea of "iterating over a range of characters".
</div>

In [ ]:
%%ipytest

def solution_str_range(start: str, end: str, step: int):
    """Return a generator from `start` to `end` strings (inclusive)"""
    

### Read `n` lines

Create a function called `read_n_lines` that takes two arguments: the filename from which to read, and the **maximum number of lines** that should be returned with each iteration.

For example, if we had a file like

```
File line 0 aaa
File line 1 bbb
File line 2 ccc
File line 3 ddd
File line 4 eee
File line 5 fff
File line 6 ggg
```

Then we could use the `read_n_lines` to read pairs of lines:

```python
for two_lines in read_n_lines(filename, 2):
    print(two_lines.rstrip())
```

And the output would be:

```
File line 0 aaa
File line 1 bbb

File line 2 ccc
File line 3 ddd

File line 4 eee
File line 5 fff

File line 6 ggg
```

The last line is returned by itself because the file has an odd number of lines.

We could also do:

```python
for four_lines in read_n(filename, 4):
    print(four_lines.rstrip())
```

And get back

```
File line 0 aaa
File line 1 bbb
File line 2 ccc
File line 3 ddd

File line 4 eee
File line 5 fff
File line 6 ggg
```

<div class="alert alert-block alert-warning">
    <h4><b>Note</b></h4> With each iteration, <code>read_n_lines</code> shoul return a string (<strong>not</strong> a list) containing up to the number of lines specified by the parameter <code>lines</code>.
</div>

In [ ]:
%%ipytest

def solution_read_n_lines(filename: str, lines: int):
    """Read multiple lines from a file"""
    

### Only run once

Create a decorator called `once` that restricts a function to run at most **once every `allowed_time` seconds**, where `allowed_time` is a parameter with a default value of `15`.

If you try to invoke the function too soon, the decorator should raise an exception called `RuntimeError` which tells you how long you need to wait before running your function again.
The error message should be `Wait another {remaining_time} seconds`, where `remaining_time` is the time left to wait before running the function again.

For example, the following code:

```python
import time

@once(15)
def hello(name):
    return f"Hello, {name}!"

for i in range(30):
    print(i)
    try:
        time.sleep(3)
        print(hello(f"attempt #{i}"))
    except TooSoonError as err:
        print(f"Too soon: {err}")
```

Should print something like:

```
0
Hello, attempt #0
1
Too soon: Wait another 12.00 seconds
2
Too soon: Wait another 8.99 seconds
3
Too soon: Wait another 5.98 seconds
4
Too soon: Wait another 2.98 seconds
5
Hello, attempt #5
6
Too soon: Wait another 12.00 seconds
```

<div class="alert alert-block alert-warning">
    <h4><b>Note</b></h4> The decorator should handle <strong>any kind</strong> of function, i.e., it should not care about the kind or number of parameters the function accepts.
</div>

<div class="alert alert-block alert-danger">
    <h4><b>Important</b></h4> The tests need to run for some time to check the solution. Don't worry if the execution of the cell below seems to be hanging: it's not.
</div>

<div class="alert alert-block alert-info">
    <h4><b>Hint</b></h4> If you are stuck, you can always comment the line <code>%%ipytest</code> to skip the tests, and enable them again when you think your solution is ready.
</div>

In [ ]:
%%ipytest
import time

def solution_once(allowed_time: int = 15) -> t.Callable:
    """Decorator to run a function at most once per given seconds"""
    